In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr

from spyfit.io._sfit4in import *
from spyfit.io._sfit4out import *

In [2]:
%cd "/home/bovy/sfit4/sfit4_v0.9.4.4/test_cases_NDACC/x.ch4/"

/home/bovy/sfit4/sfit4_v0.9.4.4/test_cases_NDACC/x.ch4


In [3]:
!du -ch --exclude '*hbin*' --exclude 'sfit4.dtl' --exclude '*.nc'

8.7M	.
8.7M	total


In [4]:
!ls

02609.659356-02908.070644.hbin	k.output	sa.complete  station.layers
ak.target			kb.out		sainv.input  summary
apod_poly.dat			pbpfile		sfit4.ctl    test.nc
aprfs.table			phase_poly.dat	sfit4.dtl
hbin.input			reference.prf	spectrum
isotope.input			rprfs.table	statevec


In [5]:
_read_g_out = lambda f: read_table(f, var_name="gain_matrix",
                                   dims=('statevector', 'diag'),
                                   index_cols=False)
_read_k_out = lambda f: read_table(f, var_name="k_matrix",
                                   dims=('diag', 'statevector'))
_read_kb_out = lambda f: read_table(f, var_name='kb_vectors',
                                    dims=('diag', 'mparam'))
_read_shat_out = lambda f: read_table(f, var_name="cov_matrix_fitted",
                                      dims=('statevector', 'statevector'))
_read_sa_out = lambda f: read_table(f, var_name="cov_matrix_initial",
                                    dims=('statevector', 'statevector'))


def _read_summary_clean(f):
    d = read_summary(f)
    # total column values may be slightly different than in
    # `file.out.retprofiles` (not the same precision): skip here.
    for k in list(d['data_vars'].keys()):
        if 'total_column' in k:
            del d['data_vars'][k]
    # wavenumber start/stop values are the same than in `sfit4.ctl`
    # but not than in `file.out.pbpfile`: skip here.
    for k in ('spec_wn', 'spec_band', 'spec_scan'):
        del d['coords'][k]
    return d


def _read_spectrum_raw(f):
    # spectral, band and scan data in `file.in.spectrum` don't
    # seem to be always equal to observed spectrum in `file.out.pbpfile`.
    # use different dimension and variable names here.
    d = read_spectrum(f, spdim='spectrum_in', bdim='band_in',
                      sdim='scan_in', wcoord='spec_in_wn',
                      scoord='spec_in_scan', bcoord='spec_in_band')
    d['data_vars'] = {k.replace('spec_', 'spec_in_'): v
                      for k, v in d['data_vars'].items()}
    return d


_map_filename_read_func = {
    # sfit4.ctl field name: (default sfit4 filename, read function) 
    'file__in__stalayers': ('station.layers', read_layers),
    'file__in__refprofile': ('reference.prf', read_ref_profiles),
    'file__in__spectrum': ('t15asc.4', _read_spectrum_raw),
    'file__out__ak_matrix': ('ak.out', read_ak_matrix),
    'file__out__seinv_vector': ('seinv.out', read_seinv_vector),
    'file__out__g_matrix': ('g.out', _read_g_out),
    'file__out__k_matrix': ('k.out', _read_k_out),
    'file__out__kb_matrix': ('kb.out', _read_kb_out),
    'file__out__shat_matrix': ('shat.complete', _read_shat_out),
    'file__out__sa_matrix': ('sa.complete', _read_sa_out),
    'file__out__aprprofiles': ('aprfs.table', read_aprfs),
    'file__out__retprofiles': ('rprfs.table', read_rprfs),
    'file__out__pbpfile': ('pbpfile', read_spectra),
    'file__out__statevec': ('statevec', read_state_vector),
    'file__out__summary': ('summary', _read_summary_clean)
}

In [6]:
#ds.sfit4_ctl

In [7]:
def _read_and_merge(ds, read_func, filename, attrs):
    temp_ds = xr.Dataset(**read_func(filename))
    ds.merge(temp_ds, inplace=True, compat='identical')
    
    # xarray doesn't merge global attributes
    attrs.update(temp_ds.attrs)


def load_sfit4_rundir(dirname):
    ds = xr.Dataset(**read_ctl(os.path.join(dirname, "sfit4.ctl")))
    global_attrs = {}
    sfit4_inputs = ds.sfit4_ctl.attrs

    for input_name, v in _map_filename_read_func.items():
        default_filename, read_func = v
        filename = sfit4_inputs.get(input_name, default_filename)
        if not os.path.exists(filename):
            continue
        _read_and_merge(ds, read_func, filename, global_attrs)
    
    if sfit4_inputs.get('out__gas_spectra', False):
        _read_and_merge(ds, read_single_spectra, 'spc.*',
                        global_attrs)
    
    # TODO: convert spectrum coord to multi-index
    
    # update/overwrite global attributes
    ds.attrs.update(global_attrs)
    ds.attrs['source'] = os.path.abspath(dirname)
    ds.attrs['description'] = 'data from a single sfit4 run'
    
    return ds

# TODO: check 'diag' dimension, same scan/band ordering than 'spectrum' dim? 

In [8]:
%timeit load_sfit4_rundir(os.curdir)

1 loop, best of 3: 1.01 s per loop


In [9]:
ds = load_sfit4_rundir(os.curdir)
ds

<xarray.Dataset>
Dimensions:                      (band: 4, band_in: 5, diag: 1610, iteration: 1, kernel: 47, level: 47, level_lbound: 48, mparam: 130, param: 8, rlevel: 43, scan: 1, scan_in: 1, spectrum: 1610, spectrum_in: 2443, statevector: 58)
Coordinates:
  * level                        (level) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 ...
  * param                        (param) <U11 'BckGrdSlp_1' 'BckGrdSlp_2' ...
    altitude                     (level) float64 113.0 100.2 89.85 81.1 ...
    spec_scan                    (spectrum) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
  * iteration                    (iteration) int64 -1
    spec_wn                      (spectrum) float64 2.614e+03 2.614e+03 ...
    spec_band                    (spectrum) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
  * spectrum                     (spectrum) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
  * level_lbound                 (level_lbound) int64 0 1 2 3 4 5 6 7 8 9 10 ...
  * band                         (band) int64 1 2 3 4
 

In [10]:
def convert_bool_attrs(dataset):
    bool2str = lambda v: 'T' if v is True else 'F' if v is False else v
    for var in dataset.variables.values():
        for k, v in var.attrs.items():
            var.attrs[k] = bool2str(v)
    for k, v in ds.attrs.items():
        ds.attrs[k] = bool2str(v)

convert_bool_attrs(ds)
encoding = {k: {'zlib': True} for k in ds.variables.keys()}
#encoding = {}

ds.to_netcdf('test.nc', encoding=encoding)

In [11]:
!du -sh test.nc

1.8M	test.nc
